# Finetune Neuralchat on NVIDIA GPU

## Prerequisite​

### Setup Environment

In [ ]:
!pip install intel-extension-for-transformers
!git clone https://github.com/intel/intel-extension-for-transformers.git
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/
!pip install -r requirements.txt
%cd ../../../
!pip uninstall torch -y
!pip install torch

### Prepare Dataset and Model

In [ ]:
data_path = "HuggingFaceH4/ultrachat_200k"
llama2_model_name_or_path = "meta-llama/Llama-2-7b-hf"

### Setup Finetuning Config

In [ ]:
from transformers import TrainingArguments
from intel_extension_for_transformers.neural_chat.config import (
    ModelArguments,
    DataArguments,
    FinetuningArguments,
    TextGenerationFinetuningConfig,
)

model_args = ModelArguments(
    model_name_or_path=llama2_model_name_or_path,
    use_fast_tokenizer=False,
)

model_args = ModelArguments(
    model_name_or_path=llama2_model_name_or_path,
    use_fast_tokenizer=False,
)


data_args = DataArguments(
    dataset_name=data_path,
    max_seq_length=1024,
    max_source_length=512,
    preprocessing_num_workers=4
)

training_args = TrainingArguments(
    output_dir="./llama_peft_finetuned_model",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=False,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    num_train_epochs=3,
    save_strategy="steps",
    save_steps=1000,
    log_level="info",
    logging_steps=10,
    save_total_limit=2,
    bf16=True,
)

finetune_args = FinetuningArguments(
    peft="lora",
    lora_alpha=64,
    lora_rank=16,
    lora_dropout=0.05,
    lora_all_linear=True,
    do_lm_eval=True,
    task="chat"
)

finetune_cfg = TextGenerationFinetuningConfig(
        model_args=model_args,
        data_args=data_args,
        training_args=training_args,
        finetune_args=finetune_args,
)

### Finetuning

In [ ]:
from intel_extension_for_transformers.neural_chat.chatbot import finetune_model
finetune_model(finetune_cfg)